# Draw dashboard to analyze dogami


## import lib

In [41]:

import pandas as pd
from pymongo import MongoClient
import plotly
from datetime import datetime, date

## define functions

In [42]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [43]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df


## testing first


In [44]:
dog_df = read_mongo(db='dogami-database', collection='dog')

dog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   token_id               8000 non-null   int64  
 1   generation             7999 non-null   object 
 2   strength               6239 non-null   float64
 3   intelligence           6239 non-null   float64
 4   breeding_count         6239 non-null   float64
 5   gender                 6239 non-null   object 
 6   size                   6239 non-null   object 
 7   status                 7999 non-null   object 
 8   friendliness           6239 non-null   float64
 9   breed                  6239 non-null   object 
 10  group                  6239 non-null   object 
 11  obedience              6239 non-null   object 
 12  rarity_tier            6239 non-null   object 
 13  eyes_color             6239 non-null   object 
 14  bonding_level          6239 non-null   float64
 15  prim

In [45]:
trans_df = read_mongo(db='dogami-database', collection='transaction')

trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2529 entries, 0 to 2528
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   token_id        2529 non-null   object
 1   operation_hash  2529 non-null   object
 2   price           2529 non-null   int64 
 3   time_stamp      2529 non-null   object
 4   order_type      2529 non-null   object
 5   trans_id        2529 non-null   object
 6   seller          1203 non-null   object
 7   buyer           1203 non-null   object
dtypes: int64(1), object(7)
memory usage: 158.2+ KB


In [46]:
dog_df['token_id'] = dog_df['token_id'].astype(str)

dog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   token_id               8000 non-null   object 
 1   generation             7999 non-null   object 
 2   strength               6239 non-null   float64
 3   intelligence           6239 non-null   float64
 4   breeding_count         6239 non-null   float64
 5   gender                 6239 non-null   object 
 6   size                   6239 non-null   object 
 7   status                 7999 non-null   object 
 8   friendliness           6239 non-null   float64
 9   breed                  6239 non-null   object 
 10  group                  6239 non-null   object 
 11  obedience              6239 non-null   object 
 12  rarity_tier            6239 non-null   object 
 13  eyes_color             6239 non-null   object 
 14  bonding_level          6239 non-null   float64
 15  prim

In [47]:
join_df = trans_df.merge(right=dog_df, how='inner', on='token_id')
join_df

,token_id,operation_hash,price,time_stamp,order_type,trans_id,seller,buyer,generation,strength,...,breed,group,obedience,rarity_tier,eyes_color,bonding_level,primary_personality,vitality,rarity_score,secondary_personality
0,7078,oo75o7qf3Lnw8aieRuxxM6YA4YU5YCwwBgYSY3UKCAgWid...,400,2022-03-16T21:14:14Z,sales,187856469,NaN,NaN,Alpha,7.0,...,Husky,Working,3,Bronze,Green #5,1.0,Gentle,11.0,46.0,Devoted
1,7078,ooFtw7MvLxzZaS7B46b5nCBrgUanCEhXqP3otVe5qfAujf...,285,2022-02-24T13:38:20Z,sales,178259619,NaN,NaN,Alpha,7.0,...,Husky,Working,3,Bronze,Green #5,1.0,Gentle,11.0,46.0,Devoted
2,3974,oo3rxMSREHkGpmYcm6FKnjUbxoBYJZuLRjKNDxKTx3UunN...,277,2022-02-23T12:15:10Z,sales,177687229,NaN,NaN,Alpha,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3809,onwfL6mB9xgsnk15mKcpE7kAswRoUiHBDGejNTtNzfskf1...,233,2022-02-23T05:10:00Z,sales,177546335,NaN,NaN,Alpha,5.0,...,German Shepherd,Herding,12,Bronze,Orange #4,1.0,Talkative,5.0,37.0,Playful
4,3809,op6NX2yaG1zAhd5dXY2iY187zSvrVPRwBywZCCnMKPY3Xz...,285,2022-03-06T14:44:34Z,sales,182998061,NaN,NaN,Alpha,5.0,...,German Shepherd,Herding,12,Bronze,Orange #4,1.0,Talkative,5.0,37.0,Playful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,539,onkUGrBPWnzWKfEeam6BSVcjUmB2CHeSVr5niHv8ieXZQJ...,400,2022-03-28T18:40:14Z,sales,193379052,tz1cREiyxjgyFDGH4JxCvj4erT1LLunanMq9,tz1eWUHK9pYC8DwsHZvPcriwUmp6C1JCYZni,Alpha,8.0,...,Husky,Working,3,Bronze,Green #5,1.0,Needy,13.0,48.0,Emotional
2520,2897,ooPr58Xak3ATfURDnUzWxytaLSbbaWjCG2bKcsb5SYZvqW...,320,2022-03-28T21:11:24Z,sales,193434904,tz1Z99n6zezGywZTvYPMgQbRvN2NUMcC5pm2,tz1iQuQ39TAxGbQjd6Zb4omfYpMtauwSvtaf,Alpha,5.0,...,Toy Poodle,Non-Sporting,6,Bronze,Brown #3,1.0,Playful,4.0,39.0,Funny
2521,7384,ooGs9ojR4Pe19uG2Jy8FuMpY1ecxzGSmQPJjmLkWJpZLk8...,10000,2022-03-29T06:05:34Z,accept_offer,193585565,tz1b2Ev1JLBN49wRcsrEhJb93pbq2wYUAiuW,tz1iELYTMCn2FeUhFSCWsi81Pe6HugLTbrhV,Alpha,8.0,...,Shiba Inu,Non-Sporting,3,Diamond,Brown #1,1.0,Protector,9.0,98.0,Independent
2522,2087,ontjTvKmJyqG1iXDWzTthJ5xyzMAhivBLXqEpMN4dQfARj...,499,2022-03-29T03:21:04Z,sales,193541252,tz1NjrNfn9bED4HkxmbL3UepgaeUJYTtzpUf,tz1R2mUXqAdJBc43JPuKrRenpbBsMaDq4Jqm,Alpha,5.0,...,German Shepherd,Herding,9,Silver,Black #2,1.0,Naive,6.0,51.0,Foodie


In [48]:
join_df['date'] = pd.to_datetime(join_df['time_stamp']).dt.date
join_df['date_string'] = join_df['date'].astype(str)
join_df

,token_id,operation_hash,price,time_stamp,order_type,trans_id,seller,buyer,generation,strength,...,obedience,rarity_tier,eyes_color,bonding_level,primary_personality,vitality,rarity_score,secondary_personality,date,date_string
0,7078,oo75o7qf3Lnw8aieRuxxM6YA4YU5YCwwBgYSY3UKCAgWid...,400,2022-03-16T21:14:14Z,sales,187856469,NaN,NaN,Alpha,7.0,...,3,Bronze,Green #5,1.0,Gentle,11.0,46.0,Devoted,2022-03-16,2022-03-16
1,7078,ooFtw7MvLxzZaS7B46b5nCBrgUanCEhXqP3otVe5qfAujf...,285,2022-02-24T13:38:20Z,sales,178259619,NaN,NaN,Alpha,7.0,...,3,Bronze,Green #5,1.0,Gentle,11.0,46.0,Devoted,2022-02-24,2022-02-24
2,3974,oo3rxMSREHkGpmYcm6FKnjUbxoBYJZuLRjKNDxKTx3UunN...,277,2022-02-23T12:15:10Z,sales,177687229,NaN,NaN,Alpha,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-23,2022-02-23
3,3809,onwfL6mB9xgsnk15mKcpE7kAswRoUiHBDGejNTtNzfskf1...,233,2022-02-23T05:10:00Z,sales,177546335,NaN,NaN,Alpha,5.0,...,12,Bronze,Orange #4,1.0,Talkative,5.0,37.0,Playful,2022-02-23,2022-02-23
4,3809,op6NX2yaG1zAhd5dXY2iY187zSvrVPRwBywZCCnMKPY3Xz...,285,2022-03-06T14:44:34Z,sales,182998061,NaN,NaN,Alpha,5.0,...,12,Bronze,Orange #4,1.0,Talkative,5.0,37.0,Playful,2022-03-06,2022-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,539,onkUGrBPWnzWKfEeam6BSVcjUmB2CHeSVr5niHv8ieXZQJ...,400,2022-03-28T18:40:14Z,sales,193379052,tz1cREiyxjgyFDGH4JxCvj4erT1LLunanMq9,tz1eWUHK9pYC8DwsHZvPcriwUmp6C1JCYZni,Alpha,8.0,...,3,Bronze,Green #5,1.0,Needy,13.0,48.0,Emotional,2022-03-28,2022-03-28
2520,2897,ooPr58Xak3ATfURDnUzWxytaLSbbaWjCG2bKcsb5SYZvqW...,320,2022-03-28T21:11:24Z,sales,193434904,tz1Z99n6zezGywZTvYPMgQbRvN2NUMcC5pm2,tz1iQuQ39TAxGbQjd6Zb4omfYpMtauwSvtaf,Alpha,5.0,...,6,Bronze,Brown #3,1.0,Playful,4.0,39.0,Funny,2022-03-28,2022-03-28
2521,7384,ooGs9ojR4Pe19uG2Jy8FuMpY1ecxzGSmQPJjmLkWJpZLk8...,10000,2022-03-29T06:05:34Z,accept_offer,193585565,tz1b2Ev1JLBN49wRcsrEhJb93pbq2wYUAiuW,tz1iELYTMCn2FeUhFSCWsi81Pe6HugLTbrhV,Alpha,8.0,...,3,Diamond,Brown #1,1.0,Protector,9.0,98.0,Independent,2022-03-29,2022-03-29
2522,2087,ontjTvKmJyqG1iXDWzTthJ5xyzMAhivBLXqEpMN4dQfARj...,499,2022-03-29T03:21:04Z,sales,193541252,tz1NjrNfn9bED4HkxmbL3UepgaeUJYTtzpUf,tz1R2mUXqAdJBc43JPuKrRenpbBsMaDq4Jqm,Alpha,5.0,...,9,Silver,Black #2,1.0,Naive,6.0,51.0,Foodie,2022-03-29,2022-03-29


In [49]:
join_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2524 entries, 0 to 2523
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   token_id               2524 non-null   object 
 1   operation_hash         2524 non-null   object 
 2   price                  2524 non-null   int64  
 3   time_stamp             2524 non-null   object 
 4   order_type             2524 non-null   object 
 5   trans_id               2524 non-null   object 
 6   seller                 1200 non-null   object 
 7   buyer                  1200 non-null   object 
 8   generation             2524 non-null   object 
 9   strength               2108 non-null   float64
 10  intelligence           2108 non-null   float64
 11  breeding_count         2108 non-null   float64
 12  gender                 2108 non-null   object 
 13  size                   2108 non-null   object 
 14  status                 2524 non-null   object 
 15  frie

In [50]:
today_date = date.today().strftime('%Y-%m-%d')
# today_date = datetime(2022, 3, 29).strftime('%Y-%m-%d')
# today_date
today_trans = join_df[join_df['date_string']==today_date]
today_trans

,token_id,operation_hash,price,time_stamp,order_type,trans_id,seller,buyer,generation,strength,...,obedience,rarity_tier,eyes_color,bonding_level,primary_personality,vitality,rarity_score,secondary_personality,date,date_string
